In [9]:
def makeBoundaries(epsilon, minValue, maxValue):
    m = epsilon * np.ceil(float(1/epsilon) * minValue)
    M = epsilon * np.floor(float(1/epsilon) * maxValue)
    return m, M

def frange(start,stop, step):
    steps = []
    while start <= stop:
        steps.append(start)
        start +=step
        
    return steps

def MakeSensorCombinations(start, end, epsilon, sensorType, room):
    a1, b1 = makeBoundaries(epsilon, start[0], end[0])
    a2, b2 = makeBoundaries(epsilon, start[1], end[1])    
    Xs = frange(a1, b1, epsilon)
    Ys = frange(a2, b2, epsilon)
    
    points = list(itertools.product(list(itertools.product(Xs, Ys)), [room], [sensorType[0]])) 
    C = itertools.combinations(points, distribution[room][types.index(sensorType)])

    return C


def SensorConfigurationSetup(mode, epsilon, rooms, types, distribution):
    sensorPlacesByRooms = {}
    sensorsCount = 0
    for room in rooms:
        sensorPlacesByRooms[room] = {}
        for sensorType in range(len(types)):
            points = SensorizeRoom(mode, rooms[room][0], rooms[room][1], epsilon, types[sensorType], room, distribution[room][sensorType])                
            sensorPlacesByRooms[room][types[sensorType][0]] = points
            sensorsCount += len(points)
            

    # print(sensorPlacesByRooms)

    columns = []
    for room in rooms:
        for sensorType in range(len(types)):
            columns.append(room + ":" + types[sensorType][0])
    
    df_ = pd.DataFrame()
    
    for room in rooms:
        for sensorType in sensorPlacesByRooms[room]:
            combinations = sensorPlacesByRooms[room][sensorType]
            tempdf = pd.DataFrame(columns=[room + ":" + sensorType])
            tempdf[room + ":" + sensorType] = [combination for combination in combinations]
            
            df_ = pd.concat([df_,tempdf], ignore_index=True, axis=1)
        
    return df_
    
def SensorizeRoom(mode, start, end, epsilon, sensorType, room, count):
    a1, b1 = makeBoundaries(epsilon, start[0], end[0])
    a2, b2 = makeBoundaries(epsilon, start[1], end[1])  

    # if (mode == 'Smart Condo Placement'):
      # if (room == 'bedroom'):

      # elif (room == 'bathroom'):


      # elif (room == 'diningroom'):


      # elif (room == 'kitchen'):


      # elif (room == 'livingroom')

    if (mode == 'Random Placement'):
      points = []
      from random import randrange

      for i in range(count):
        X = randrange(a1 * 10, b1 * 10) / 10
        Y = randrange(a2 * 10, b2 * 10) / 10
        points.append((X, Y))

      points = list(itertools.product(points, [room], [sensorType[0]])) 

    elif (mode == 'Bad Placement'):
      points = []
      from random import randrange

      for i in range(count):
        X = randrange(a1 * 10, (a1 + 0.5) * 10) / 10
        Y = randrange(a2 * 10, (a2 + 0.5) * 10) / 10
        points.append((X, Y))

      points = list(itertools.product(points, [room], [sensorType[0]])) 

    return points

def ConvertE_ToDict(df_, rooms, distribution, types, roomsList, sensorPlaces):
    dictE = {}
    for sensorType in range(len(types)):
        prev = 0
        dictE[types[sensorType][0]] = {}
        A = df_[types[sensorType][0]].to_numpy()
        
        E = np.empty([len(A), len(A[0])])
        for i in range(len(A)):
            E[i] = np.array(A[i])
        
        for roomIndex in range(len(roomsList)):
            dictE[types[sensorType][0]][roomsList[roomIndex]] = []
            sensorsNum = len(sensorPlaces[roomsList[roomIndex]][types[sensorType][0]])
            sensorsNum = sensorsNum + prev
            dictE[types[sensorType][0]][roomsList[roomIndex]] = E[:, prev : sensorsNum]
            prev = sensorsNum
                        
    return dictE

def DeploySimulation():
    simworldname = './Data/simulationWorld2.xml'
    space, rooms = pf.ParseWorld(simworldname)
    types, distribution = pf.GetUsersParameters()

    roomsList = []

    for room in distribution:
        roomsList.append(room)
        
    epsilon = 1
    iteration = 1

    # SCS = MakeSensorConfigurationCombinations(epsilon)
    SCS, allSensorPositionsCount, sensorPlaces = SensorConfigurationSetup(epsilon, rooms, types)
    
    # Args = []
    # for c in range(len(SCS.columns)):
        # Args.append((SCS.loc[SCS[c].notnull(), SCS.columns[c]]))

    # args = []
    # for i in range(len(Args)):
    #     for j in range(len(Args[i])):
    #         args.append(Args[i][j])

    # args = tuple(args)
    
    import copy
    types2 = copy.deepcopy(types)

    types2[0][1] = int(allSensorPositionsCount)
    types2[1][1] = int(allSensorPositionsCount)
    
    
    iteration = 1
    df_ = sim_sis.RunSimulator(space, rooms, [types2, [args]], True, True, False, iteration)
    
    return space, rooms, types, distribution, roomsList, SCS, allSensorPositionsCount, sensorPlaces, df_
    
    
def CreateSensorEventsDictionary(space, rooms, types, distribution, roomsList, SCS, sensorPlaces, df_):
    dictE = ConvertE_ToDict(df_, rooms, distribution, types, roomsList, sensorPlaces)
    return dictE

def CreateRoomCombinations(roomX, roomName):
    brTemp = []
    brCombinations = {}
    for i in range(len(roomX)):
        brTemp.append(itertools.combinations(range(len(roomX[i][0])), distribution[roomName][0]))

    for i in range(len(brTemp)):
        if i == 0: 
            brCombinations['motion sensors'] = []
            for c in brTemp[i]:
                brCombinations['motion sensors'].append(c)

        elif i == 1: 
            brCombinations['beacon sensors'] = []
            for c in brTemp[i]:
                brCombinations['beacon sensors'].append(c)

    return brCombinations

def EvaluateConfigurations(dictE, r1Name, r2Name, r1Combinations, r2Combinations, sensorName):
    import sys
    import SIM_SIS_Libraries.PreDeploymentEvaluation as pde

    inter = len(r1Combinations[sensorName])
    taskNum = (len(r1Combinations[sensorName]) * len(r2Combinations[sensorName]))
    for x, y in itertools.product(range(len(r1Combinations[sensorName])), range(len(r2Combinations[sensorName]))):
        if x % 100 == 0:
            sys.stderr.write('\r %.2f percent completed' % float((x * 100)/(inter)))

        pde.ConfigurationEvaluation([(dictE[sensorName][r1Name][:,r1Combinations[sensorName][x]],  
                                      dictE[sensorName][r2Name][:,r2Combinations[sensorName][y]])]) 

    print("Done!")

In [10]:
if __name__ ==  '__main__':
    from google.colab import drive
    import sys
    drive.mount('/content/gdrive', force_remount=True)
    Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
    sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
    import SIM_SIS_Libraries.SensorsClass
    import SIM_SIS_Libraries.SIM_SIS_Simulator as sim_sis
    import SIM_SIS_Libraries.ParseFunctions as pf
    import itertools
    import numpy as np
    import pandas as pd
    import sys
    import SIM_SIS_Libraries.PreDeploymentEvaluation as pde
    import copy
    import os
    
    # Space model path
    simworldname = Data_path + '/Configuration Files/simulationWorld2.xml'

    agentTraces = []
    directory = os.fsencode(Data_path + 'Agent Trace Files/')
        
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            agentTraces.append(Data_path + 'Agent Trace Files/' + filename)

    # Parsing the space model: 
    space, rooms = pf.ParseWorld(simworldname)
    # User parameters 
    types, distribution = pf.GetUsersParameters()

    roomsList = []

    for room in distribution:
        roomsList.append(room)
        
    epsilon = 1
    radius = 1
    hypithesisConfigurations = 10
    mode = 'Bad Placement'
    # mode = 'Smart Condo Placement'
    # mode = 'Random Placement'


    for cc in range(hypithesisConfigurations):
      iteration = 1
      SCdf = SensorConfigurationSetup(mode, epsilon, rooms, types, distribution)
        
      Args = []
      for c in range(len(SCdf.columns)):
          Args.append((SCdf.loc[SCdf[c].notnull(), SCdf.columns[c]]))

      args = []
      for i in range(len(Args)):
          for j in range(len(Args[i])):
              args.append(Args[i][j])

      args = tuple(args)

      sensorConfiguration = [[types, [args]], radius]
      simulateMotionSensors = True
      simulateEstimotes = False
      Plotting = False

      
      for agentTrace in agentTraces:
        print('----- Running simulation for:' + agentTrace)
        filecoding = "_" + str(iteration) + "_c" + str(cc + 1) + '(' + mode + ')'
        df_ = sim_sis.RunSimulator(space, rooms, agentTrace, sensorConfiguration, simulateMotionSensors, simulateEstimotes, Plotting, filecoding , Data_path)
        iteration += 1
      
    
    
    
    
    
    

Mounted at /content/gdrive
----- Running simulation for:gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/Agent Trace Files/Agent 1.csv
----- Running simulation for:gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/Agent Trace Files/Agent 2.csv
----- Running simulation for:gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/Agent Trace Files/Agent 3.csv
----- Running simulation for:gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/Agent Trace Files/Agent 1.csv
----- Running simulation for:gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/Agent Trace Files/Agent 2.csv
----- Running simulation for:gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/Agent Trace Files/Agent 3.csv
----- Running simulation for:gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/Agent Trace Files/Agent 1.csv
----- Running simulation for:gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/Agent Trace Files/Agent 2.csv
----- Running simulation for:gdrive/My Drive/PhD/Thes

In [3]:
distribution

{'bathroom': [2, 2],
 'bedroom': [3, 3],
 'diningroom': [2, 2],
 'kitchen': [4, 4],
 'livingroom': [3, 3]}